Lecture 4
===========================================================================

One dimensional unconstrained optimisation
------------

In [20]:
from sympy import *
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline
init_printing(use_latex='mathjax')

Bracketing
------------
We seek an interval on which the minimum occurs. We search with acceleration, doubling the interval size each time until the function no longer descends.

In [ ]:
x = var('x')
f = (x - 100)**2
xint = [0, 1] # initial interval
delta = 1
k = 1
while f.subs(x,xint[k]) <  f.subs(x,xint[k-1]):
    xint.append(xint[k] + delta*2**k)
    k = k + 1
    
for c1,c2 in zip(xint, [f.subs(x,xi) for xi in xint]):
    print "%-9.8s %0.8s" % (c1,c2)

The minimum must lie between the third last and last x value in the table above.

In [ ]:
fval = [f.subs(x,xi) for xi in xint]
evalfunc = lambdify(x, f, modules=['numpy'])
xval = np.linspace(0.0, 300.0, 1000)
plt.plot(xval, evalfunc(xval))
plt.scatter(xint, fval, s=50, color='r')
plt.xlim(0,300)
plt.show()

Now we restart the search on the new interval

In [ ]:
xint = [xint[np.size(xint)-3],xint[np.size(xint)-3]+1] # initial interval
delta = 1
k = 1
while f.subs(x,xint[k]) <  f.subs(x,xint[k-1]):
    xint.append(xint[k] + delta*2**k)
    k = k + 1
    
fval = [f.subs(x,xi) for xi in xint]
for c1,c2 in zip(xint, fval):
    print "%-9s %s" % (c1,c2)

Now the minimum is between 85 and 109, to further refine the search we can to reduce the intervals by reducing delta

In [ ]:
evalfunc = lambdify(x, f, modules=['numpy'])
xval = np.linspace(0.0, 300.0, 1000)
plt.plot(xval, evalfunc(xval))
plt.scatter(xint, fval, s=50, color='r')
plt.xlim(0,300)
plt.show()

Newton's Method
------------

In [ ]:
x = var('x')
f = x**4 - x + 1
fd = f.diff(x)
fdd = f.diff(x, 2)
fd, fdd

In [ ]:
N = 10
xs = [3.0]
for i in range(N-1): # For convenience we do a fixed number of points
    xs.append(xs[-1] - fd.subs(x,xs[-1])/fdd.subs(x,xs[-1]))

Nfs = [f.subs(x,xi) for xi in xs]
xsN = xs
for c1,c2 in zip(xsN, Nfs):
    print "%-12.9s %0.9s" % (c1,c2)

How does it work?

Newton's method starts off by using a second order Taylor expansion, this is equivalent to approximating the function with a quadratic polynomial:

In [ ]:
evalfunc = lambdify(x, f, modules=['numpy'])
xval = np.linspace(-5.0, 5.0, 100)
tfix = series(f, x, x0=3, n=None)
taylor = sum([next(tfix) for _ in range(3)])
evaltaylor = lambdify(x, taylor, modules=['numpy'])
xval2 = np.linspace(-0.0, 5.0, 100)
plt.plot(xval, evalfunc(xval))
plt.plot(xval2, evaltaylor(xval2), color='r')
plt.scatter(3.0, evalfunc(3.0), s=100, color='magenta')
plt.show()

Then the derivative of this quadratic function is set to zero, this is equivalent to approximating the true derivative at the point as a straight line and finding its intercept with the x-axis:

In [ ]:
fd = f.diff(x)
evalfuncd = lambdify(x, fd, modules=['numpy'])
taylord = taylor.diff(x)
evaltaylord = lambdify(x, taylord, modules=['numpy'])
plt.plot(xval, evalfuncd(xval))
plt.plot(xval2, evaltaylord(xval2), color='r')
plt.scatter(3.0, evalfuncd(3.0), s=100, color='magenta')
plt.scatter(3.0, evalfuncd(3.0), s=100, color='magenta')
plt.scatter(solve(taylord)[0], evalfuncd(solve(taylord)[0]), s=100, color='green')
plt.axvline(0, color='black')
plt.axhline(0, color='black')
plt.axvline(solve(taylord)[0], color='green')
plt.xlim(-2,5)
plt.ylim(-100,300)
plt.show()

This gives a value of around 2, as indicated in the table above and is also equivalent to jumping to minimum of the previously fitted quadratic. This is used as the new x value to evaluate the slope.

In [ ]:
from IPython.html.widgets import interact, interactive
from IPython.html import widgets
from IPython.display import display

In [ ]:
def show_newton(niter):
    x = var('x')
    f = x**4 - x + 1
    fd = f.diff(x)
    fdd = f.diff(x, 2)
    N = 10
    xs = [3.0]
    for i in range(N-1):
        xs.append(xs[-1] - fd.subs(x,xs[-1])/fdd.subs(x,xs[-1]))
    Nfs = [f.subs(x,xi) for xi in xs]
    xsN = xs
    evalfunc = lambdify(x, f, modules=['numpy'])
    xval = np.linspace(-5.0, 5.0, 100)
    tfix = series(f, x, x0=xsN[niter], n=None)
    taylor = sum([next(tfix) for _ in range(3)])
    evaltaylor = lambdify(x, taylor, modules=['numpy'])
    xval2 = np.linspace(-0.0, 5.0, 100)    
    evalfuncd = lambdify(x, fd, modules=['numpy'])
    taylord = taylor.diff(x)
    evaltaylord = lambdify(x, taylord, modules=['numpy'])
    plt.plot(xval, evalfuncd(xval))
    plt.plot(xval2, evaltaylord(xval2), color='r')
    plt.scatter(xsN[niter], evalfuncd(xsN[niter]), s=100, color='magenta')
    plt.scatter(solve(taylord)[0], evalfuncd(solve(taylord)[0]), s=100, color='green')
    plt.axvline(0, color='black')
    plt.axhline(0, color='black')
    plt.axvline(solve(taylord)[0], color='green')
    plt.xlim(-2,3)
    plt.ylim(-50,50)
    plt.show()

In [ ]:
from collections import OrderedDict

In [ ]:
testValues = [1,2,3,4,5,6,7]
testValuesn = ['1','2','3','4','5','6','7']
testValues = OrderedDict(zip(testValuesn, testValues))
ne = interactive(show_newton, niter=testValues)
display(ne)

Quasi-Newton (Secant) method
------------

Note that this variation is a bracketing method, but secant methods do not require the points to span the mnimum in general. The algorithm implemented here is as it is described in the textbook.


In [ ]:
# Secant method on this function
N = 10 # iterations
xp = -3.0
xq = 3.0 # initial points
xps = []
xqs = []
xs = []
for i in range(N):
    xps.append(xp)
    xqs.append(xq)
    fxq = fd.subs(x,xq)
    fxp = fd.subs(x,xp)
    xroot = (xq - fxq * (xq - xp) / (fxq - fxp))
    xs.append(xroot)
    froot = fd.subs(x,xroot)
    if sign(fxp) == sign(froot):
        xp = xroot
    else:
        xq = xroot

Sfs = [f.subs(x,xi) for xi in xs]
xsS = xs
for c1,c2 in zip(xsS, Sfs):
    print "%-12.9s %0.9s" % (c1,c2)

The method effectively searches for the zero intercept of the derivative as approximated as a straight line between the two chosen points.

In [ ]:
m = (evalfuncd(-3.0) - evalfuncd(3.0))/(-3.0 - 3.0)
c = evalfuncd(3.0) - 3.0 * m
line = m*x + c
evalline = lambdify(x, line, modules=['numpy'])
plt.plot(xval, evalline(xval), color='r')
plt.plot(xval, evalfuncd(xval), color='b')
plt.scatter(-3.0, evalfuncd(-3.0), s=100, color='magenta')
plt.scatter(3.0, evalfuncd(3.0), s=100, color='magenta')
plt.scatter(solve(line)[0], evalfuncd(solve(line)[0]), s=100, color='green')
plt.axvline(0, color='black')
plt.axhline(0, color='black')
plt.axvline(solve(line)[0], color='green')
plt.xlim(-4,4)
plt.ylim(-300,300)
plt.show()

This is equivalent to finding the minimum of a quadratic with the same slopes as the function at the these two points.

In [ ]:
fapprox = line.integrate(x)
evalfapprox = lambdify(x, fapprox, modules=['numpy'])
plt.plot(xval, evalfunc(xval))
plt.plot(xval, evalfapprox(xval), color='r')
plt.scatter(solve(line)[0], evalfunc(solve(line)[0]), s=100, color='green')
plt.scatter(-3.0, evalfunc(-3.0), s=100, color='magenta')
plt.scatter(3.0, evalfunc(3.0), s=100, color='magenta')
plt.show()

One of the remaining points must be selected in order to calculate the new approximation, this is done by ensuring that the slope has the opposite sign at the two chosen points.

In [ ]:
def show_secant(niter):
    x = var('x')
    f = x**4 - x + 1
    fd = f.diff(x)
    fdd = f.diff(x, 2)
    N = 10
    xp = -3.0
    xq = 3.0 # initial points
    xps = []
    xqs = []
    xs = [-3.0]
    for i in range(N):
        xps.append(xp)
        xqs.append(xq)
        fxq = fd.subs(x,xq)
        fxp = fd.subs(x,xp)
        xroot = (xq - fxq * (xq - xp) / (fxq - fxp))
        xs.append(xroot)
        froot = fd.subs(x,xroot)
        if sign(fxp) == sign(froot):
            xp = xroot
        else:
            xq = xroot
    Sfs = [f.subs(x,xi) for xi in xs]
    xsS = xs
    m = (evalfuncd(xsS[niter-1]) - evalfuncd(3.0))/(xsS[niter-1] - 3.0)
    c = evalfuncd(3.0) - 3.0 * m
    line = m*x + c
    evalline = lambdify(x, line, modules=['numpy'])
    plt.plot(xval, evalline(xval), color='r')
    plt.plot(xval, evalfuncd(xval), color='b')
    plt.scatter(xsS[niter-1], evalfuncd(xsS[niter-1]), s=100, color='magenta')
    plt.scatter(3.0, evalfuncd(3.0), s=100, color='magenta')
    plt.scatter(solve(line)[0], evalfuncd(solve(line)[0]), s=100, color='green')
    plt.axvline(0, color='black')
    plt.axhline(0, color='black')
    plt.axvline(solve(line)[0], color='green')
    plt.xlim(-4,4)
    plt.ylim(-300,300)
    plt.show()

In [ ]:
se = interactive(show_secant, niter=testValues)
display(se)

Secant starts better but converges slower for this function. This is in fact a general rule.

In [ ]:
iterval = np.linspace(1.0, 11.0, 10)
Nfsarr = np.asarray(Nfs)
Nfslog = [log(y+1) for y in Nfsarr]
Sfsarr = np.asarray(Sfs)
Sfslog = [log(y+1) for y in Sfsarr]
plt.scatter(iterval, Nfslog, s=100, color='g')
plt.scatter(iterval, Sfslog, s=100, color='b')
plt.show()